# Experimenting With APIs

In this notebook, we will be experimenting with the APIs and create helper functions.

In [1]:
# general imports
import time
import re
from collections import namedtuple

# my imports
from helpers import kube

## Kubernetes API

We use the kubernetes API to fetch the current status of each deployment and see how many instances they have.

In [2]:
# start watch thread
kube.start_watch_thread()
# print details of a deployment
# print(kube.live_deployments['helloworld-go-00001-deployment'])
# fetch a list of all knative deployments
# print(kube.get_knative_deployments())
# fetch latest revisions only

def get_knative_watch_info(kn_deploy_name):
    kn_latest_revs = kube.get_kn_latest_revs()
    if kn_deploy_name not in kn_latest_revs:
        return None
    bench_deployment = kn_latest_revs[kn_deploy_name]
    return kube.live_deployments[bench_deployment.deployment]

get_knative_watch_info('bench1')

starting watch thread...


## Deploying On Knative

In this section, we will develop the functionality to deploy a given workload to **knative** using the `kn` CLI.
The other options were using [CRDs](https://stackoverflow.com/questions/61384188/how-to-deploy-a-knative-service-with-kubernetes-python-client-library),
or using [kubectl apply](https://developers.redhat.com/coderland/serverless/deploying-serverless-knative#invoking_your_service_from_the_command_line)
but `kn` seems to be more powerful.

In [3]:
# making sure kn is set up correctly
!kn service ls

NAME            URL                                            LATEST                AGE   CONDITIONS   READY   REASON
helloworld-go   http://helloworld-go.default.kn.nima-dev.com   helloworld-go-00001   24h   3 OK / 3     True    


In [6]:
name = 'bench1'
image = 'ghcr.io/nimamahmoudi/conc-workloads-bench1:sha-5966a0e'
env = {
    'EXPERIMENT_NAME': 'TEST1',
    'REPORT_INTERVAL': '10',
    'SOCKETIO_SERVER': 'NO',
}
opts = {
    '--limit': "'cpu=250m,memory=256Mi'",
    '--concurrency-target': '1',
    # '--concurrency-limit': '10',
    # '--concurrency-utilization': '70',
    # '--autoscale-window': '60s',
}
annotations = {
    'autoscaling.knative.dev/panicThresholdPercentage': '1000',
}
kn_command = kube.get_kn_command(name, image, env, opts, annotations)
print(kn_command)
# to run the command, we can simply use:
# !{kn_command}

kn service apply bench1 --image ghcr.io/nimamahmoudi/conc-workloads-bench1:sha-5966a0e \
  --env EXPERIMENT_NAME=TEST1 \
  --env REPORT_INTERVAL=10 \
  --env SOCKETIO_SERVER=NO \
  --limit 'cpu=250m,memory=256Mi' \
  --concurrency-target 1 \
  -a autoscaling.knative.dev/panicThresholdPercentage=1000
